In [2]:
import napari
import numpy as np
import tifffile as tiff
import zarr
import dask.array as da
from dask.diagnostics import ProgressBar
import pandas as pd
import xml.etree.ElementTree as ET
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# def open_image_napari(image_path, markers):
#     # create a colormap list with the same length as markers - ['cyan', 'green', 'yellow', 'red', 'magenta', 'orange', 'blue' ] repeat until reach len(markers)
#     colormap = ['cyan', 'green', 'yellow', 'red', 'magenta', 'orange', 'blue']
#     colormap = colormap * (len(markers) // len(colormap) + 1)

#     store = tiff.imread(image_path, aszarr=True)
#     cache = zarr.LRUStoreCache(store, max_size=2**30)
#     zobj = zarr.open(cache, mode='r')
#     data = [
#         zobj[int(dataset['path'])]
#         for dataset in zobj.attrs['multiscales'][0]['datasets']
#     ]
#     data = [da.from_zarr(z) for z in data]
#     n_channels = data[0].shape[0]
#     for z in data:
#         print(z)
#     viewer = napari.Viewer()
#     # Add each channel as a separate image layer
#     for i in range(n_channels):
#         channel_data = [data_layer[i] for data_layer in data]  # Extract the i-th channel
#         viewer.add_image(
#             channel_data,
#             name=markers.marker_name[i],
#             rgb=False,
#             contrast_limits=[0, 65535],
#             blending='additive',
#             colormap=colormap[i],
#         )
#     napari.run()
#     store.close()

#     return viewer

In [7]:
def open_multiseries_image_napari(image_path:str, markers:list) -> napari.Viewer:
    colormap = ['cyan', 'green', 'yellow', 'red', 'magenta', 'orange', 'blue']
    colormap = colormap * (len(markers) // len(colormap) + 1)

    pyramid = []
    with tiff.TiffFile(image_path) as tif:
        for series in tif.series:
            store = series.aszarr()
            dask_array = da.from_zarr(store)
            pyramid.append(dask_array)
    n_channels = pyramid[0].shape[0]
    viewer = napari.Viewer()
    for i in range(n_channels):
        channel_data = [data_layer[i] for data_layer in pyramid]
        viewer.add_image(
            channel_data,
            name=markers[i],
            multiscale=True,
            contrast_limits=[0, 65535],
            blending='additive',
            colormap=colormap[i],
        )
    napari.run()
    store.close()

    return viewer
    

In [27]:
def open_multiseries_image_napari(image_path:str) -> napari.Viewer:
    colormap = ['cyan', 'green', 'yellow', 'red', 'magenta', 'orange', 'blue']
   

    pyramid = []
    with tiff.TiffFile(image_path) as tif:
        for series in tif.series:
            store = series.aszarr()
            dask_array = da.from_zarr(store)
            pyramid.append(dask_array)
    n_channels = pyramid[0].shape[0]
    colormap = colormap * (n_channels // len(colormap) + 1)
    viewer = napari.Viewer()
    for i in range(n_channels):
        channel_data = [data_layer[i] for data_layer in pyramid]
        viewer.add_image(
            channel_data,
            name=f'Channel {i+1}',
            multiscale=True,
            contrast_limits=[0, 65535],
            blending='additive',
            colormap=colormap[i],
        )
    napari.run()
    store.close()

    return viewer
    

In [3]:
image_path = '/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/img/S011_chemonaive.ome.tif'
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/markers.txt') as f:
    channel_names = [line.strip() for line in f.readlines()]

In [8]:
open_multiseries_image_napari(image_path, channel_names)

Viewer(camera=Camera(center=(0.0, np.float64(6709.0), np.float64(12045.5)), zoom=np.float64(0.04749605678233438), angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True, orientation=(<DepthAxisOrientation.TOWARDS: 'towards'>, <VerticalAxisOrientation.DOWN: 'down'>, <HorizontalAxisOrientation.RIGHT: 'right'>)), cursor=Cursor(position=(1.0, 1.0), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=2, ndisplay=2, order=(0, 1), axis_labels=('0', '1'), rollable=(True, True), range=(RangeTuple(start=np.float64(0.0), stop=np.float64(13418.0), step=np.float64(1.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(24091.0), step=np.float64(1.0))), margin_left=(0.0, 0.0), margin_right=(0.0, 0.0), point=(np.float64(6709.0), np.float64(12045.0)), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False, spacing=0.0), layers=[<Image layer 'DNA1' at 0x318067ee0>, <Image layer 'Rabbit' at 0x31d96c1c0>, <Image layer 'Goat' at 0x31eddff10>,

In [3]:
def print_metadata_structure(element, indent=0):
    print("  " * indent + f"Tag: {element.tag}")
    if element.attrib:
        print("  " * (indent + 1) + "Attributes:")
        for key, value in element.attrib.items():
            print("  " * (indent + 2) + f"{key}: {value}")
    for child in element:
        print_metadata_structure(child, indent + 1)

In [ ]:
store = tiff.imread(image_path, aszarr=True)
cache = zarr.LRUStoreCache(store, max_size=2**30)
zobj = zarr.open(cache, mode='r')